# 1. Libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import os

# 2. Load data

In [2]:
df_nivel = pd.read_csv('/home/nivelrios/documentos/Mburicao Project/data/interim/nivel.csv')

# 3. Processing

In [3]:
df_nivel_copy = df_nivel.copy()
df_nivel_copy['fecha'] = pd.to_datetime(df_nivel_copy['fecha'], errors='coerce')
df_nivel_copy['fecha'] = df_nivel_copy['fecha'].dt.round('S')
df_nivel_copy['fecha'] = df_nivel_copy['fecha'].dt.round('T')
df_nivel_copy['nivel'] = df_nivel_copy['nivel'].round(4)

# 4. Save dataset

In [4]:
ruta = "/home/nivelrios/documentos/Mburicao Project/data/processed"
os.makedirs(ruta, exist_ok=True)
archivo = os.path.join(ruta, "nivel_processed.csv")
df_nivel_copy.to_csv(archivo, index=False)

# 5. Peak detection

In [74]:
fechas = df_nivel_copy['fecha']
y = df_nivel_copy['nivel'].values

###################################################################
#   Detección de inicio y fin de un evento dado un umbral 0.3     #
###################################################################

UMBRAL = 0.3
indices_inicio = np.where((y[:-1] < UMBRAL) & (y[1:] >= UMBRAL))[0] + 1
indices_fin = np.where((y[:-1] >= UMBRAL) & (y[1:] < UMBRAL))[0] + 1

###################################################################
#       Emparejar un inicio y con el primer inmediato fin         #
###################################################################

eventos = []
for inicio in indices_inicio:
    posibles_fines = indices_fin[indices_fin > inicio]
    if posibles_fines.size > 0:
        fin = posibles_fines[0]
        eventos.append((inicio, fin))

###################################################################
#          Extracción de información de los eventos               #
###################################################################

eventos_list = []
for inicio, fin in eventos:
    segmento = y[inicio:fin]  # Extraemos la porción del evento
    picos, _ = find_peaks(segmento)  # Detectamos picos locales en el segmento
    duracion_evento = (fechas.iloc[fin] - fechas.iloc[inicio]).total_seconds() / 60.0 # Duración total del evento (en minutos)

    
    local_val = None
    local_fecha = None
    duracion_local = None
    global_val = None
    global_fecha = None
    duracion_global = None

    DIFERENCIA = 0.4 # Condición para pico local: debe tener (valor del pico - valor inicio) >= 0.4
    if picos.size > 0:
        pico_local_idx_global = None
        for candidate in picos:
            candidate_global_idx = inicio + candidate
            if (y[candidate_global_idx] - y[inicio]) >= DIFERENCIA:
                pico_local_idx_global = candidate_global_idx
                break
        
        if pico_local_idx_global is not None:
            local_val = y[pico_local_idx_global]
            local_fecha = fechas.iloc[pico_local_idx_global]
            duracion_local = (local_fecha - fechas.iloc[inicio]).total_seconds() / 60.0

        pico_global_idx_segmento = picos[np.argmax(segmento[picos])] # Pico global: el pico con el valor máximo en el segmento (sin la restricción de 0.4)
        pico_global_idx_global = inicio + pico_global_idx_segmento

        global_val = y[pico_global_idx_global]
        global_fecha = fechas.iloc[pico_global_idx_global]
        duracion_global = (global_fecha - fechas.iloc[inicio]).total_seconds() / 60.0

    evento_info = {
        "inicio": fechas.iloc[inicio],
        "fin": fechas.iloc[fin],
        "local": local_val,
        "local_fecha": local_fecha,
        "global": global_val,
        "global_fecha": global_fecha,
        "duracion_evento": duracion_evento,
        "duracion_local": duracion_local,
        "duracion_global": duracion_global
    }
    eventos_list.append(evento_info)

df_eventos = pd.DataFrame(eventos_list)

# 6. Save dataset

In [77]:
ruta = "/home/nivelrios/documentos/Mburicao Project/data/external"
os.makedirs(ruta, exist_ok=True)
archivo = os.path.join(ruta, "peak_detection_info.csv")
df_eventos.to_csv(archivo, index=False)